In [2]:
import pandas as pd
import numpy as np
import obspy
from obspy.core import Stream, Trace
from obspy.signal.trigger import classic_sta_lta, trigger_onset
import os
import datetime
import math # Necesitamos importar math para ceil

# --- 1. Entrada de parámetros del usuario ---
while True:
    file_path = input("Por favor, introduce la ruta COMPLETA de tu archivo TXT de datos sísmicos sin comillas (ej. F:\\ruta\\al\\archivo.txt): ")
    if os.path.exists(file_path):
        print(f"Ruta de archivo '{file_path}' encontrada. Procediendo...")
        break
    else:
        print(f"Error: La ruta '{file_path}' no existe o no se puede acceder.")
        print("Asegúrate de escribir la ruta correctamente y de que el archivo exista.")

while True:
    try:
        sampling_rate_str = input("Introduce la frecuencia de muestreo de tus datos en Hz (ej. 40): ")
        sampling_rate = float(sampling_rate_str)
        if sampling_rate <= 0:
            print("La frecuencia de muestreo debe ser un número positivo.")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la frecuencia de muestreo.")

# --- Solicitar fecha y hora de inicio ---
while True:
    start_date_str = input("Introduce la fecha de inicio de la muestra (YYYY-MM-DD, ej. 2023-01-15): ")
    start_time_str = input("Introduce la hora de inicio de la muestra (HH:MM:SS.ffffff, ej. 10:30:00.000000): ")
    try:
        start_datetime = datetime.datetime.strptime(f"{start_date_str} {start_time_str}", "%Y-%m-%d %H:%M:%S.%f")
        print(f"Fecha y hora de inicio configuradas: {start_datetime}")
        break
    except ValueError:
        print("Formato de fecha u hora inválido. Por favor, usa %Y-%m-%d y HH:MM:SS.ffffff.")

while True:
    try:
        freq_min_bandpass_str = input("Introduce la frecuencia INFERIOR del filtro de paso de banda en Hz (ej. 1.0): ")
        freq_min_bandpass = float(freq_min_bandpass_str)
        if freq_min_bandpass < 0:
            print("La frecuencia mínima no puede ser negativa.")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la frecuencia mínima.")

while True:
    try:
        freq_max_bandpass_str = input("Introduce la frecuencia SUPERIOR del filtro de paso de banda en Hz (ej. 5.0): ")
        freq_max_bandpass = float(freq_max_bandpass_str)
        if freq_max_bandpass <= freq_min_bandpass:
            print("La frecuencia superior debe ser mayor que la frecuencia inferior.")
        elif freq_max_bandpass >= (sampling_rate / 2):
            print(f"Advertencia: La frecuencia superior ({freq_max_bandpass} Hz) es igual o mayor que la frecuencia de Nyquist ({sampling_rate / 2} Hz).")
            print("Esto puede llevar a resultados inesperados. Considera una frecuencia superior menor.")
            if input("¿Deseas continuar de todos modos? (s/n): ").lower() != 's':
                continue
            break
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la frecuencia superior.")

# --- Parámetro para unificación de componentes ---
while True:
    unify_components_str = input("¿Deseas unificar las componentes E, N, Z en una sola columna 'Magnitude'? (s/n): ").lower()
    if unify_components_str in ['s', 'n']:
        unify_components = True if unify_components_str == 's' else False
        break
    else:
        print("Respuesta inválida. Por favor, introduce 's' para sí o 'n' para no.")

# --- Umbral de magnitud para filtrar ---
magnitude_threshold = None
if unify_components:
    while True:
        try:
            magnitude_threshold_str = input("Introduce el umbral de magnitud (ej. 0.1) para filtrar la columna unificada/filtrada: ")
            magnitude_threshold = float(magnitude_threshold_str)
            if magnitude_threshold < 0:
                print("El umbral de magnitud no puede ser negativo.")
            else:
                break
        except ValueError:
            print("Entrada inválida. Por favor, introduce un número para el umbral de magnitud.")
else:
    print("Nota: No se solicitará umbral de magnitud ya que las componentes no se unificarán.")

# --- Duración mínima del evento en segundos ---
while True:
    try:
        min_event_duration_str = input("Introduce la duración mínima de un evento (en segundos) para ser registrado (ej. 0.5): ")
        min_event_duration = float(min_event_duration_str)
        if min_event_duration < 0:
            print("La duración mínima no puede ser negativa.")
        else:
            break
    except ValueError:
        print("Entrada inválida. Por favor, introduce un número para la duración mínima.")


# --- Parámetros fijos para STA/LTA y filtro ---
corners = 4
stalta_short = 1.0
stalta_long = 10.0 # Ventana LTA de 10 segundos
thresh_on = 2.5
thresh_off = 0.8


# --- Cálculo dinámico del tamaño del chunk ---
print("\nCalculando el tamaño de los bloques para dividir el archivo en 50 partes...")
try:
    with open(file_path, 'r') as f:
        total_lines = sum(1 for line in f)
    
    if total_lines == 0:
        raise ValueError("El archivo de datos está vacío.")

    # Calcula el tamaño del chunk para que haya 50 partes, usando ceil para asegurar que se cubran todas las líneas
    chunk_size = int(math.ceil(total_lines / 50))
    
    # Asegura que el chunk_size no sea cero si el archivo es muy pequeño (menos de 50 líneas)
    if chunk_size == 0: 
        chunk_size = total_lines # Si hay menos de 50 líneas, se procesa todo en un chunk.

    print(f"El archivo tiene {total_lines} registros.")
    print(f"Se dividirá en 50 bloques de aproximadamente {chunk_size} registros cada uno.")

    # --- ADVERTENCIA CRÍTICA SOBRE EL TAMAÑO DEL CHUNK Y STA/LTA ---
    min_samples_for_stalta_LTA_window = int(stalta_long * sampling_rate)
    # Recomendamos que el chunk sea al menos 2-3 veces el tamaño de la ventana LTA para que funcione bien
    min_recommended_chunk_for_stalta = int(min_samples_for_stalta_LTA_window * 2.5) 

    if chunk_size < min_recommended_chunk_for_stalta:
        print(f"\n¡¡¡ADVERTENCIA CRÍTICA!!!")
        print(f"El tamaño de cada bloque ({chunk_size} muestras) es MUY PEQUEÑO para la ventana larga de STA/LTA ({stalta_long}s = {min_samples_for_stalta_LTA_window} muestras).")
        print(f"Esto causará que el algoritmo STA/LTA NO SE EJECUTE CORRECTAMENTE o NO DETECTE EVENTOS en la mayoría de los bloques.")
        print(f"Se recomienda que el tamaño del bloque sea al menos {min_recommended_chunk_for_stalta} muestras para que STA/LTA funcione.")
        print("Considera dividir el archivo en MENOS bloques (ej. 5 o 10 en lugar de 50) o ajustar tus ventanas STA/LTA (stalta_long).")
        
        while True:
            confirm = input("¿Deseas continuar a pesar de esta advertencia? (s/n): ").lower()
            if confirm == 's':
                break
            elif confirm == 'n':
                print("Procesamiento cancelado por el usuario.")
                exit() # Sale del script
            else:
                print("Respuesta inválida. Por favor, introduce 's' o 'n'.")

except Exception as e:
    print(f"Error al calcular el tamaño total de líneas o el tamaño del chunk: {e}")
    print("Asegúrate de que la ruta del archivo sea correcta y que el archivo no esté vacío.")
    exit()


print(f"\n--- Configuración final ---")
print(f"  Archivo TXT: {file_path}")
print(f"  Frecuencia de muestreo: {sampling_rate} Hz")
print(f"  Fecha y hora de inicio: {start_datetime}")
print(f"  Filtro de paso de banda: {freq_min_bandpass}-{freq_max_bandpass} Hz")
print(f"  Unificar componentes: {'Sí' if unify_components else 'No'}")
if unify_components:
    print(f"  Umbral de magnitud para filtro: {magnitude_threshold}")
print(f"  Duración mínima de evento para registro: {min_event_duration}s")
print(f"  Tamaño de bloque para procesamiento: {chunk_size} muestras (para 50 partes)")
print(f"  Ventana STA/LTA Corta: {stalta_short}s")
print(f"  Ventana STA/LTA Larga: {stalta_long}s")
print(f"  Umbral STA/LTA ON: {thresh_on}")
print(f"  Umbral STA/LTA OFF: {thresh_off}")

print("\n--- Procesando el archivo... ---")

# --- Cargar datos y crear columna 'timestamp' ---
try:
    start_time_current_chunk = start_datetime
    
    # Lista para almacenar los DataFrames de chunks procesados
    all_processed_chunks_for_final_df = [] 
    # Lista para almacenar los detalles de los eventos detectados (para usar en el filtrado final)
    detected_event_summary_list = []

    # Iterar a través del archivo por chunks
    for i, chunk_df in enumerate(pd.read_csv(file_path, sep='\s+', header=None, names=['E', 'N', 'Z'], chunksize=chunk_size)):
        print(f"\nProcesando bloque {i+1} de aproximadamente {int(np.ceil(total_lines / chunk_size))}...")
        
        # Calcular el timestamp para el chunk actual
        chunk_df['timestamp'] = [start_time_current_chunk + datetime.timedelta(seconds=j / sampling_rate) 
                                 for j in range(len(chunk_df))]
        
        # Actualizar el tiempo de inicio para el siguiente chunk
        start_time_current_chunk = chunk_df['timestamp'].iloc[-1] + datetime.timedelta(seconds=1 / sampling_rate)

        # --- Unificar componentes si se solicitó ---
        if unify_components:
            chunk_df['Magnitude'] = np.sqrt(chunk_df['E']**2 + chunk_df['N']**2 + chunk_df['Z']**2)
            data_to_filter = chunk_df['Magnitude'].values
        else:
            data_to_filter = chunk_df['E'].values 

        # --- Filtrar pasa-banda ---
        trace = Trace(data=data_to_filter)
        trace.stats.sampling_rate = sampling_rate
        trace.filter('bandpass', freqmin=freq_min_bandpass, freqmax=freq_max_bandpass, corners=corners, zerophase=True)
        
        # Asignar los datos filtrados de vuelta al DataFrame
        if unify_components:
            chunk_df['Magnitude_Filtered'] = trace.data
        else:
            chunk_df['E_Filtered'] = trace.data

        # --- Filtrar el DataFrame por el umbral de magnitud unificada (si aplica) ---
        if unify_components and magnitude_threshold is not None:
            current_chunk_processed = chunk_df[chunk_df['Magnitude_Filtered'] > magnitude_threshold].copy()
        else:
            current_chunk_processed = chunk_df.copy() 

        # Añadir el chunk procesado a la lista para el DataFrame final
        if not current_chunk_processed.empty:
            all_processed_chunks_for_final_df.append(current_chunk_processed)

        # --- Preparar datos para STA/LTA ---
        if unify_components:
            data_for_stalta = current_chunk_processed['Magnitude_Filtered'].values
        else:
            data_for_stalta = current_chunk_processed['E_Filtered'].values
            
        # --- Aplicar STA/LTA ---
        if data_for_stalta.size > 0:
            npts_long = int(stalta_long * sampling_rate)
            
            if len(data_for_stalta) < npts_long:
                # Esta advertencia es normal aquí si el chunk_size es pequeño. La advertencia principal ya se dio arriba.
                pass 
            else:
                cft = classic_sta_lta(data_for_stalta, int(stalta_short * sampling_rate), int(stalta_long * sampling_rate))

                on_off = trigger_onset(cft, thresh_on, thresh_off)
                
                if on_off.size > 0:
                    source_df_for_timestamps = current_chunk_processed

                    for onset, offset in on_off:
                        if onset < len(source_df_for_timestamps) and offset < len(source_df_for_timestamps):
                            start_time_event = source_df_for_timestamps['timestamp'].iloc[onset]
                            end_time_event = source_df_for_timestamps['timestamp'].iloc[offset]
                            event_duration = (end_time_event - start_time_event).total_seconds()
                            
                            if event_duration >= min_event_duration:
                                detected_event_summary_list.append({
                                    'Start_Time': start_time_event,
                                    'End_Time': end_time_event,
                                    'Duration_s': event_duration,
                                })

except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no fue encontrado. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error durante el procesamiento: {e}")

# --- Concatenar todos los chunks procesados en un único DataFrame base ---
final_processed_df = pd.DataFrame()
if all_processed_chunks_for_final_df:
    print("\n--- Concatenando todos los bloques procesados en un DataFrame base... ---")
    final_processed_df = pd.concat(all_processed_chunks_for_final_df).reset_index(drop=True)
    print(f"DataFrame base procesado tiene {len(final_processed_df)} filas.")
else:
    print("\nNo hay datos procesados para formar el DataFrame base (posiblemente todos fueron filtrados por magnitud o archivo vacío).")


# --- Filtrar el DataFrame base para generar el archivo final con datos dentro de los eventos ---
if not final_processed_df.empty and detected_event_summary_list:
    print("\n--- Generando el ÚNICO archivo de salida con datos sísmicos dentro de los eventos detectados... ---")
    
    events_df_summary = pd.DataFrame(detected_event_summary_list)
    event_data_rows_for_final_output = []

    # Iterar a través de cada evento detectado para extraer los datos correspondientes
    for index, event in events_df_summary.iterrows():
        event_start = event['Start_Time']
        event_end = event['End_Time']
        event_duration = event['Duration_s']

        # Seleccionar las filas de final_processed_df que caen dentro del rango de tiempo de este evento
        data_in_event = final_processed_df[
            (final_processed_df['timestamp'] >= event_start) & 
            (final_processed_df['timestamp'] <= event_end)
        ].copy() 

        if not data_in_event.empty:
            # Añadir las columnas de información del evento a estas filas
            data_in_event['Inicio_Evento'] = event_start
            data_in_event['Fin_Evento'] = event_end
            data_in_event['Duracion_Evento_s'] = event_duration
            
            # Decidir qué columna de magnitud/filtrada incluir y renombrarla si es necesario
            if unify_components:
                data_in_event['Magnitud_Filtrada_MI'] = data_in_event['Magnitude_Filtered']
                cols_to_keep = ['E', 'N', 'Z', 'timestamp', 'Magnitud_Filtrada_MI', 'Inicio_Evento', 'Fin_Evento', 'Duracion_Evento_s']
            else:
                data_in_event['E_Filtrada'] = data_in_event['E_Filtered'] 
                cols_to_keep = ['E', 'N', 'Z', 'timestamp', 'E_Filtrada', 'Inicio_Evento', 'Fin_Evento', 'Duracion_Evento_s']

            # Seleccionar y reordenar las columnas deseadas para el archivo final
            data_in_event = data_in_event[cols_to_keep]
            event_data_rows_for_final_output.append(data_in_event)

    if event_data_rows_for_final_output:
        final_event_data_df = pd.concat(event_data_rows_for_final_output).reset_index(drop=True)
        # Nombre del único archivo de salida
        output_event_data_filename = os.path.splitext(file_path)[0] + "_datos_eventos_detectados.csv"
        final_event_data_df.to_csv(output_event_data_filename, index=False)
        print(f"\nEl ÚNICO archivo de salida con los datos sísmicos filtrados de los eventos detectados se ha guardado en:\n{output_event_data_filename}")
    else:
        print("\nNo se encontraron datos sísmicos dentro de los eventos detectados que cumplan todos los criterios después de los filtros.")
else:
    print("\nNo se pudo generar el archivo de datos de eventos filtrados (no se detectaron eventos o el DataFrame base está vacío).")

print("\n--- Procesamiento completado ---")

<>:190: SyntaxWarning: invalid escape sequence '\s'
<>:190: SyntaxWarning: invalid escape sequence '\s'
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_3476\696250649.py:190: SyntaxWarning: invalid escape sequence '\s'
  for i, chunk_df in enumerate(pd.read_csv(file_path, sep='\s+', header=None, names=['E', 'N', 'Z'], chunksize=chunk_size)):


Por favor, introduce la ruta COMPLETA de tu archivo TXT de datos sísmicos sin comillas (ej. F:\ruta\al\archivo.txt):  F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt


Ruta de archivo 'F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt' encontrada. Procediendo...


Introduce la frecuencia de muestreo de tus datos en Hz (ej. 40):  40
Introduce la fecha de inicio de la muestra (YYYY-MM-DD, ej. 2023-01-15):  2015-02-01
Introduce la hora de inicio de la muestra (HH:MM:SS.ffffff, ej. 10:30:00.000000):  00:00:00.000000


Fecha y hora de inicio configuradas: 2015-02-01 00:00:00


Introduce la frecuencia INFERIOR del filtro de paso de banda en Hz (ej. 1.0):  1.0
Introduce la frecuencia SUPERIOR del filtro de paso de banda en Hz (ej. 5.0):  5.0
¿Deseas unificar las componentes E, N, Z en una sola columna 'Magnitude'? (s/n):  S
Introduce el umbral de magnitud (ej. 0.1) para filtrar la columna unificada/filtrada:  0.1
Introduce la duración mínima de un evento (en segundos) para ser registrado (ej. 0.5):  7



Calculando el tamaño de los bloques para dividir el archivo en 50 partes...
El archivo tiene 96768200 registros.
Se dividirá en 50 bloques de aproximadamente 1935364 registros cada uno.

--- Configuración final ---
  Archivo TXT: F:\gustavo\Desktop\PracticasProfecionalizantes_1_2025\TRVA.201502.E.N.Z.txt
  Frecuencia de muestreo: 40.0 Hz
  Fecha y hora de inicio: 2015-02-01 00:00:00
  Filtro de paso de banda: 1.0-5.0 Hz
  Unificar componentes: Sí
  Umbral de magnitud para filtro: 0.1
  Duración mínima de evento para registro: 7.0s
  Tamaño de bloque para procesamiento: 1935364 muestras (para 50 partes)
  Ventana STA/LTA Corta: 1.0s
  Ventana STA/LTA Larga: 10.0s
  Umbral STA/LTA ON: 2.5
  Umbral STA/LTA OFF: 0.8

--- Procesando el archivo... ---

Procesando bloque 1 de aproximadamente 50...

Procesando bloque 2 de aproximadamente 50...

Procesando bloque 3 de aproximadamente 50...

Procesando bloque 4 de aproximadamente 50...

Procesando bloque 5 de aproximadamente 50...

Procesando b